In [8]:
import pandas as pd
import numpy as np


from neuralforecast import NeuralForecast
from neuralforecast.models import (
    NBEATS,
    NHITS,
    TFT)

from statsforecast.core import StatsForecast
from neuralforecast.auto import AutoMLP, AutoDeepAR, AutoNBEATS, AutoNHITS, AutoTFT , AutoDeepNPTS
from statsforecast.models import (
    Naive,
    SeasonalNaive,
    ARIMA,
    SimpleExponentialSmoothing,
    SimpleExponentialSmoothingOptimized,
    SeasonalExponentialSmoothing,
    SeasonalExponentialSmoothingOptimized,
    RandomWalkWithDrift,
    ETS,
    HistoricAverage,
    WindowAverage,
    AutoARIMA,
    AutoETS,
    AutoCES,
    AutoTheta
)
from statsforecast.models import SimpleExponentialSmoothing
# Naive
# Naive season
# Seasonal Arima (0,1 1)( 0,1,1)

from mlforecast import MLForecast
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

import lightgbm as lgb
from mlforecast.target_transforms import Differences
from mlforecast.lag_transforms import ExpandingMean, RollingMean
from numba import njit
from window_ops.rolling import rolling_mean

import time
import plotly.graph_objects as go


In [9]:
def evaluate_forecast(y_true, y_pred):
    return np.sqrt(np.mean((y_true.values - y_pred.values) ** 2))

def smape(y_true, y_pred):
    return 100 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)) / 2)

def rmsse(y_true, y_pred, train):
    naive_forecast = np.roll(train, 1)
    naive_forecast[0] = train[0]
    scale = np.mean((train - naive_forecast) ** 2)
    return np.sqrt(np.mean((y_true - y_pred) ** 2) / scale)

def calculate_errors(test, forecasts, train):

    df = pd.DataFrame()
    for col in forecasts.columns:
        error_dict = {}
        
        if col in ['ds', 'unique_id']:
            continue
        else:
            y_true = test['y'].values
            y_pred = forecasts[col].values
            error_dict.update({
                f'RMSE': evaluate_forecast(test['y'], forecasts[col]),
                f'SMAPE': smape(y_true, y_pred),
                f'RMSSE': rmsse(y_true, y_pred, train['y'].values)
            })
        df = pd.concat([pd.DataFrame(error_dict, index=[col]),df])
    return df

In [10]:

from statsforecast.utils import AirPassengersDF

# Load the AirPassengers dataset
df = AirPassengersDF

forecast_horizon = [24, 3,6,12]

# Train-test split
test_size_total = 24

train_size_total = len(df) - test_size_total
train_total, test_total = df[:train_size_total], df[train_size_total:]


df.to_csv('Air_passengers.csv')

In [16]:
train

,unique_id,ds,y
0,1.0,1949-01-31,112.0
1,1.0,1949-02-28,118.0
2,1.0,1949-03-31,132.0
3,1.0,1949-04-30,129.0
4,1.0,1949-05-31,121.0
...,...,...,...
115,1.0,1958-08-31,505.0
116,1.0,1958-09-30,404.0
117,1.0,1958-10-31,359.0
118,1.0,1958-11-30,310.0


In [29]:

models = [
    Naive(),
    SeasonalNaive(12),
    ARIMA(order=[12,1,0]),
    ARIMA(order=[0,1,1], seasonal_order=[0,1,1], season_length=12 ,alias='SARIMA'),
    SimpleExponentialSmoothing(alpha=0.28),
    ETS(model='AAA',season_length=12, alias='ETS AAA'),
    ETS(model='MAM',season_length=12, alias='ETS MAM'),
    ETS(model='MMM',season_length=12, alias='ETS MMM'),
    ETS(model='MAM',season_length=12, alias='ETS MAdM',damped=True),
    HistoricAverage(),
    WindowAverage(window_size=6),
    AutoARIMA(max_p=12),
    AutoETS(season_length=12),
    AutoETS(season_length=12,damped=True,alias='Damped AutoETS'),
    AutoCES(season_length=12,alias='AutoCES'),
    AutoTheta(season_length=12),
    SimpleExponentialSmoothingOptimized(),
    SeasonalExponentialSmoothing(season_length=12,alpha=0.28),
    SeasonalExponentialSmoothingOptimized(season_length=12),
    RandomWalkWithDrift(),
]

error_dfs = []

for model in models:
    for horizon in forecast_horizon:
        df_forecast_model = pd.DataFrame()
        total_train_time = 0
        combined_forecasts = pd.DataFrame()

        for start in range(0, test_size_total, horizon):
            end = start + horizon
            train_size = train_size_total + start
            train = df[:train_size]
            test = df[train_size:train_size + horizon]
            sf = StatsForecast(
                models=[model],
                freq='ME',
                n_jobs=-1,
            )

            start_time = time.time()
            forecasts_df = sf.forecast(df=train, h=horizon)
            train_time = time.time() - start_time

            forecasts_df['origin'] = train_size  # Track the forecast origin point
            combined_forecasts = pd.concat([combined_forecasts, forecasts_df])
            total_train_time += train_time
        
            
        # Calculate errors for the combined forecast
        combined_error_statsforecast = calculate_errors(test_total, combined_forecasts.drop(columns=['origin']), train_total)
        
        combined_error_statsforecast['Total_Train_Time'] = total_train_time
        combined_error_statsforecast['Horizon'] = horizon
        combined_error_statsforecast['Model'] = str(model)
        error_dfs.append(combined_error_statsforecast)

# Combine all errors into a single DataFrame
Error_statstical = pd.concat(error_dfs).reset_index(drop=True)

c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\statsforecast\models.py:880: FutureWarning: `ETS` will be deprecated in future versions of `StatsForecast`. Please use `AutoETS` instead.
  ETS._warn()
c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\statsforecast\core.py:485: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\statsforecast\core.py:485: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\statsforecast\core.py:485: FutureWarning: In a future version the predictions will have the id

In [6]:
@njit
def rolling_mean_12(x):
    return rolling_mean(x, window_size=12)


def month_index(times):
    return times.month

In [7]:
lgb_params = {'verbosity': -1,'num_leaves': 512,}

catboost_params ={'subsample': 0.6 , 'iterations': 50, 'depth': 5, 'verbose':0}

xgboost_params ={'verbosity':0, 'max_depth':5 , 'subsample': 0.6}

randomforest_params = {'verbose': 0, 'max_depth': 5}
models={
        'LightGBM': lgb.LGBMRegressor(**lgb_params),
        'CatBoost': CatBoostRegressor(**catboost_params),
        'XgBoost': XGBRegressor(**xgboost_params),
        'RandomForest': RandomForestRegressor(**randomforest_params)
    }


In [8]:

lgb_params = {'verbosity': -1,'num_leaves': 512,}

catboost_params ={'subsample': 0.6 , 'iterations': 50, 'depth': 5, 'verbose':0}

xgboost_params ={'verbosity':0, 'max_depth':5 , 'subsample': 0.6}

randomforest_params = {'verbose': 0, 'max_depth': 5}
models={
        'LightGBM': lgb.LGBMRegressor(**lgb_params),
        'CatBoost': CatBoostRegressor(**catboost_params),
        'XgBoost': XGBRegressor(**xgboost_params),
        'RandomForest': RandomForestRegressor(**randomforest_params)
    }

for alias,model in models.items():
    for horizon in forecast_horizon:
        df_forecast_model = pd.DataFrame()
        total_train_time = 0
        combined_forecasts = pd.DataFrame()

        for start in range(0, test_size_total, horizon):
            end = start + horizon
            train_size = train_size_total + start
            train = df[:train_size]
            test = df[train_size:train_size + horizon]
            
            fcst = MLForecast(
                models = {alias:model,},
                freq="ME",
                target_transforms=[Differences([12])],    
                lags= [1,2,3,4,11,12],
                lag_transforms={
                    1: [ExpandingMean()],
                    12: [RollingMean(window_size=12), rolling_mean_12],
                },
                date_features=[month_index],
            )

            start_time = time.time()
            prep = fcst.preprocess(train)
            fcst.fit(train)
            forecasts_df = fcst.predict(h=horizon)
            train_time = time.time() - start_time

            forecasts_df['origin'] = train_size  # Track the forecast origin point
            combined_forecasts = pd.concat([combined_forecasts, forecasts_df])
            total_train_time += train_time
        
            
        # Calculate errors for the combined forecast
        combined_error_statsforecast = calculate_errors(test_total, combined_forecasts.drop(columns=['origin']), train_total)
        
        combined_error_statsforecast['Total_Train_Time'] = total_train_time
        combined_error_statsforecast['Horizon'] = horizon
        combined_error_statsforecast['Model'] = alias
        error_dfs.append(combined_error_statsforecast)

# Combine all errors into a single DataFrame
Error_Tree = pd.concat(error_dfs).reset_index(drop=True)

In [9]:
Error_Tree

,RMSE,SMAPE,RMSSE,Total_Train_Time,Horizon,Model
0,137.328985,6.937759,4.801815,0.000000,24,Naive
1,62.552311,2.837492,2.187191,0.033325,3,Naive
2,77.368167,3.688349,2.705239,0.016682,6,Naive
3,108.203127,4.838893,3.783407,0.016819,12,Naive
4,76.994589,4.253156,2.692176,0.000000,24,SeasonalNaive
...,...,...,...,...,...,...
59,16.963066,0.835843,0.593127,0.250320,12,XgBoost
60,36.636748,1.943429,1.281033,0.282578,24,RandomForest
61,23.390363,1.046343,0.817862,1.158152,3,RandomForest
62,23.278571,1.149863,0.813953,0.682989,6,RandomForest


In [5]:
from ray import tune
from ray import tune
neural_models = [
    NBEATS(input_size=2 * test_size_total, h=test_size_total,),
    NHITS(input_size=2 * test_size_total, h=test_size_total,),
    AutoMLP(config=dict(input_size=tune.choice([3 * test_size_total]), learning_rate=tune.choice([1e-3])), h=test_size_total, num_samples=1, cpus=3,verbose=False),
    AutoDeepAR(config=dict( input_size=tune.choice([3 * test_size_total]), learning_rate=tune.choice([1e-3])), h=test_size_total, num_samples=1, cpus=3),
    AutoNBEATS(config=dict( input_size=tune.choice([3 * test_size_total]), learning_rate=tune.choice([1e-3])), h=test_size_total, num_samples=1, cpus=3),
    AutoNHITS(config=dict(input_size=tune.choice([3 * test_size_total]), learning_rate=tune.choice([1e-3])), h=test_size_total, num_samples=1, cpus=3),
    AutoTFT(config=dict( input_size=tune.choice([3 * test_size_total]), learning_rate=tune.choice([1e-3])), h=test_size_total, num_samples=1, cpus=3)
]



for horizon in forecast_horizon:
    for model in neural_models:
        total_train_time = 0
        combined_forecasts = pd.DataFrame()
        for start in range(0, test_size_total, horizon):
            end = start + horizon
            train_size = train_size_total + start
            train = df[:train_size]
            test = df[train_size:train_size + horizon]

            nf = NeuralForecast(models=[model], freq='ME')

            start_time = time.time()
            nf.fit(df=train)
            forecasts_df_neural = nf.predict().reset_index()
            train_time = time.time() - start_time
            forecasts_df_neural = forecasts_df_neural[:horizon]
            forecasts_df_neural['origin'] = train_size  # Track the forecast origin point
            combined_forecasts = pd.concat([combined_forecasts, forecasts_df_neural])
            total_train_time += train_time

        # Calculate errors for the combined forecast
        combined_error_statsforecast = calculate_errors(test_total, combined_forecasts.drop(columns=['origin']), train_total)
        combined_error_statsforecast['Total_Train_Time'] = total_train_time
        combined_error_statsforecast['Horizon'] = horizon
        combined_error_statsforecast['Model'] = type(model).__name__ + '_old'
        error_dfs.append(combined_error_statsforecast)

# Combine all errors into a single DataFrame
all_errors_old = pd.concat(error_dfs).reset_index(drop=True)
print(all_errors_old)


Seed set to 1
Seed set to 1
c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\pytorch_lightning\utilities\parsing.py:199: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.


In [1]:
# error_dfs=[]
# forecast_horizon = [24, 3]
# for model in neural_models:
#     for horizon in forecast_horizon:
#         total_train_time = 0
#         combined_forecasts = pd.DataFrame()
#         for start in range(0, test_size_total, horizon):
#             end = start + horizon
#             train_size = train_size_total + start
#             train = df[:train_size]
#             test = df[train_size:train_size + horizon]

#             nf = NeuralForecast(models=[model], freq='ME')

#             start_time = time.time()
#             nf.fit(df=train,verbose=False)
#             forecasts_df_neural = nf.predict().reset_index()
#             train_time = time.time() - start_time
#             forecasts_df_neural = forecasts_df_neural[:horizon]
#             forecasts_df_neural['origin'] = train_size  # Track the forecast origin point
#             combined_forecasts = pd.concat([combined_forecasts, forecasts_df_neural])
#             total_train_time += train_time

#         # Calculate errors for the combined forecast
#         combined_error_statsforecast = calculate_errors(test_total, combined_forecasts.drop(columns=['origin']), train_total)
#         combined_error_statsforecast['Total_Train_Time'] = total_train_time
#         combined_error_statsforecast['Horizon'] = horizon
#         combined_error_statsforecast['Model'] = type(model).__name__
#         error_dfs.append(combined_error_statsforecast)

# # Combine all errors into a single DataFrame
# all_errors_old = pd.concat(error_dfs).reset_index(drop=True)
# print(all_errors_old)

NameError: name 'neural_models' is not defined

In [6]:

for horizon in forecast_horizon:
    neural_models = [
      NBEATS(input_size=2 * horizon, h=horizon,),
      NHITS(input_size=2 * horizon, h=horizon,),
      AutoMLP(config=dict(input_size=tune.choice([3 * horizon]),      learning_rate=tune.choice([1e-3])), h=horizon, num_samples=1,verbose=False),
      AutoDeepAR(config=dict( input_size=tune.choice([3 * horizon]), learning_rate=tune.choice([1e-3])), h=horizon, num_samples=1,),
      AutoNBEATS(config=dict( input_size=tune.choice([3 * horizon]), learning_rate=tune.choice([1e-3])), h=horizon, num_samples=1,),
      AutoNHITS(config=dict(input_size=tune.choice([3 * horizon]), learning_rate=tune.choice([1e-3])), h=horizon, num_samples=1,),
      AutoTFT(config=dict( input_size=tune.choice([3 * horizon]), learning_rate=tune.choice([1e-3])), h=horizon, num_samples=1,)
    ]
    for model in neural_models:
        total_train_time = 0
        combined_forecasts = pd.DataFrame()
        for start in range(0, test_size_total, horizon):
            end = start + horizon
            train_size = train_size_total + start
            train = df[:train_size]
            test = df[train_size:train_size + horizon]

            nf = NeuralForecast(models=[model], freq='ME')

            start_time = time.time()
            nf.fit(df=train)
            forecasts_df_neural = nf.predict().reset_index()
            train_time = time.time() - start_time
            forecasts_df_neural = forecasts_df_neural[:horizon]
            forecasts_df_neural['origin'] = train_size  # Track the forecast origin point
            combined_forecasts = pd.concat([combined_forecasts, forecasts_df_neural])
            total_train_time += train_time

        # Calculate errors for the combined forecast
        combined_error_statsforecast = calculate_errors(test_total, combined_forecasts.drop(columns=['origin']), train_total)
        combined_error_statsforecast['Total_Train_Time'] = total_train_time
        combined_error_statsforecast['Horizon'] = horizon
        combined_error_statsforecast['Model'] = type(model).__name__ + 'new'
        error_dfs.append(combined_error_statsforecast)

# Combine all errors into a single DataFrame
all_errors_new = pd.concat(error_dfs).reset_index(drop=True)
print(all_errors_new)

(_train_tune pid=296) c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\ray\tune\integration\pytorch_lightning.py:198: `ray.tune.integration.pytorch_lightning.TuneReportCallback` is deprecated. Use `ray.tune.integration.pytorch_lightning.TuneReportCheckpointCallback` instead.
(_train_tune pid=296) Seed set to 1
(_train_tune pid=296) c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\neuralforecast\common\_base_model.py:208: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
(_train_tune pid=296)   warnings.warn(
(_train_tune pid=296) GPU available: False, used: False
(_train_tune pid=296) TPU available: False, using: 0 TPU cores
(_train_tune pid=296) IPU available: False, using: 0 IPUs
(_train_tune pid=296) HPU available: False, using: 0 HPUs
(_train_tune pid=296) Missing logger folder: C:\Users\91976\AppData\Local\Temp\ray\session_2024-07-08_23-22-47_905608_23472\artifacts\2024-07-08_23-31-36\_train_tune

Epoch 48:   0%|          | 0/1 [00:00<?, ?it/s, v_num=0, train_loss_step=25.20, train_loss_epoch=25.20]        


(_train_tune pid=296) `Trainer.fit` stopped: `max_steps=50` reached.
2024-07-08 23:31:51,581	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-07-08 23:31:51,597	INFO tune.py:1007 -- Wrote the latest version of all result files and experiment state to 'C:/Users/91976/ray_results/_train_tune_2024-07-08_23-31-36' in 0.0164s.
Seed set to 1
c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\neuralforecast\common\_base_model.py:208: UserWarning: val_check_steps i

Epoch 49: 100%|██████████| 1/1 [00:00<00:00, 15.02it/s, v_num=0, train_loss_step=23.50, train_loss_epoch=23.90]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 49: 100%|██████████| 1/1 [00:00<00:00, 11.86it/s, v_num=0, train_loss_step=23.50, train_loss_epoch=23.50, valid_loss=48.10]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=50` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

         RMSE     SMAPE     RMSSE  Total_Train_Time  Horizon    Model
0   24.693348  1.148987  0.863422         13.741171       24   NBEATS
1   34.433476  1.601430  1.203993          7.992301       24    NHITS
2   48.286379  2.278759  1.688371         24.125031       24  AutoMLP
3   28.495339  1.374399  0.996362         64.277226        3   NBEATS
4   31.581240  1.512609  1.104263         70.448647        3    NHITS
5   46.132508  2.210183  1.613059        170.937955        3  AutoMLP
6   23.478878  1.073148  0.820957         32.797676        6   NBEATS
7   29.770980  1.398068  1.040966         35.078066        6    NHITS
8   46.805428  2.289865  1.636588         78.213762        6  AutoMLP
9   27.997106  1.314268  0.978941         16.047240       12   NBEATS
10  36.905528  1.655785  1.290431         15.909688       12    NHITS
11  42.607187  2.080357  1.489794         40.210559       12  AutoMLP


c:\Users\91976\Desktop\Forecast_llm_comparison\.env\lib\site-packages\neuralforecast\core.py:196: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [28]:
from nixtla import NixtlaClient
nixtla_client = NixtlaClient(
    api_key = 'nixtla-tok-BWWtvgUP9FLtzerA90xyzXPvRUoZvA0OYYp5cuSI7NZUyApQjlINlF8dAyYXqDyxWlTlCOg7jXHWJV4o'
)

In [29]:
# error_dfs = []
for horizon in forecast_horizon:
    df_forecast_model = pd.DataFrame()
    total_train_time = 0
    combined_forecasts = pd.DataFrame()

    for start in range(0, test_size_total, horizon):
        end = start + horizon
        train_size = train_size_total + start
        train = df[:train_size]
        test = df[train_size:train_size + horizon]

        start_time = time.time()
        forecasts_df = nixtla_client.forecast(df=train, h=horizon, freq='M', time_col='ds', target_col='y')
        train_time = time.time() - start_time

        forecasts_df['origin'] = train_size  # Track the forecast origin point
        combined_forecasts = pd.concat([combined_forecasts, forecasts_df])
        total_train_time += train_time
    
        
    # Calculate errors for the combined forecast
    combined_error_statsforecast = calculate_errors(test_total, combined_forecasts.drop(columns=['origin']), train_total)
    
    combined_error_statsforecast['Total_Train_Time'] = total_train_time
    combined_error_statsforecast['Horizon'] = horizon
    combined_error_statsforecast['Model'] = "TimeGPT"
    error_dfs.append(combined_error_statsforecast)

# Combine all errors into a single DataFrame
Error_TimeGPT = pd.concat(error_dfs).reset_index(drop=True)

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: ME
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: ME
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: ME
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: ME
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.

In [35]:

# Sort the DataFrame by the 'Error' column
sorted_df = Error_TimeGPT.sort_values(by='RMSSE')



sorted_df.to_csv('results.csv')
